Import packages 

In [16]:
import re 
import sys
import string
import csv 
import sqlite3
import tqdm 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
import nltk 
from nltk.corpus import stopwords 
import numpy as np
from uszipcode import SearchEngine, SimpleZipcode, Zipcode

Increase CSV field size limit to maximim possible

In [17]:
field_size_limit = sys.maxsize
while True:
    try:
        csv.field_size_limit(field_size_limit)
        break
    except OverflowError:
        field_size_limit = int(field_size_limit / 10)

Read in data and create SQL database to store data 

In [18]:
# Read Data 
calc_data = []
with open ('calc_scrape070421.csv') as df: 
    jd = csv.reader(df)    
    next(jd)
    for i in jd: 
        calc_data.append((int(i[0]),i[15],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10],int(i[11]),float(i[12]),i[13],i[14],i[16],i[17],i[18]))


sql_string1 = """
    CREATE TABLE IF NOT EXISTS calc(
        c_row_id                     INTEGER, 
        contract_id                  TEXT,  
        business_size                TEXT,
        schedule                     TEXT, 
        site                         TEXT, 
        begin_date                   TEXT, 
        end_date                     TEXT, 
        SIN                          TEXT, 
        vendor_name                  TEXT, 
        c_labor category             TEXT, 
        c_edu_level                  TEXT, 
        c_min_year_exp               INTEGER, 
        c_current_year               REAL, 
        c_next_year                  TEXT, 
        c_second_year                TEXT, 
        pdf_url                      TEXT, 
        c_jd                         TEXT, 
        zip_code                     INTEGER 
        ); 
    """ 

allwyn_data = []
with open('Allwyn Initial Job Description Input.csv') as df: 
    a_jd = csv.reader(df)
    next(a_jd)
    for i in a_jd: 
        allwyn_data.append((int(i[0]),i[1],int(i[3]),i[4],i[6]))

print(i[4] for i in allwyn_data)    

sql_string2 = """
    CREATE TABLE IF NOT EXISTS allwyn(
        a_row_id                    INTEGER, 
        a_labor category            TEXT, 
        a_year_exp                  INTEGER, 
        a_edu_level                 TEXT, 
        a_jd                        TEXT  
        ); 
    """

# create database 
calc_con = sqlite3.Connection('calc.db')
c = calc_con.cursor()

c.execute("DROP TABLE IF EXISTS calc")
c.execute(sql_string1)
c.executemany("INSERT INTO calc VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);", calc_data)
calc_con.commit()  

c.execute("DROP TABLE IF EXISTS allwyn")
c.execute(sql_string2)
c.executemany("INSERT INTO allwyn VALUES(?,?,?,?,?);", allwyn_data)
calc_con.commit()

<generator object <genexpr> at 0x13814e820>


Create pendas dataframe for NLP 

In [19]:
rd = c.execute("SELECT c_row_id, c_labor category, c_jd, c_min_year_exp FROM calc;").fetchall()
c_jd_nlp = [(x[0],x[1],x[2],x[3]) for x in rd]

rd = c.execute("SELECT a_labor category, a_jd FROM allwyn;").fetchall()
a_jd_nlp = [(x[0],x[1]) for x in rd]

c_df = pd.DataFrame(c_jd_nlp, columns=['c_id','c_title','c_jd','c_min_year'])
a_df = pd.DataFrame(a_jd_nlp, columns=['a_title','a_jd'])

# Drop rows without calc_jd
c_df.replace('', np.nan, inplace=True) 
c_df = c_df.dropna(subset = ['c_jd'])

# NLP 

Job title matching 

In [20]:
count_vect = CountVectorizer()
X_train_counts_title = count_vect.fit_transform(a_df['a_title'])
X_train_counts_title.shape
count_vect.vocabulary_.get(u'algorithm')

index_title = count_vect.get_feature_names() 

tfidf_transformer = TfidfTransformer()
X_train_tfidf_title = tfidf_transformer.fit_transform(X_train_counts_title)
X_train_tfidf_title.shape

a_tfidf_title = X_train_tfidf_title.toarray() 

clf_title = MultinomialNB().fit(X_train_tfidf_title, a_df['a_title'])

docs_new_title = c_df['c_title']
X_new_counts_title = count_vect.transform(docs_new_title)
X_new_tfidf_title = tfidf_transformer.transform(X_new_counts_title)

c_tfidf_title = X_new_tfidf_title.toarray()

# cosine similarity score
sc_score_title = cosine_similarity(c_tfidf_title,a_tfidf_title)
print(max(sc_score_title[11])) 

predicted_title = clf_title.predict(X_new_tfidf_title)

text_clf_title = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])
text_clf_title.fit(a_df['a_jd'], a_df['a_title'])

0.5773502691896257


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

Job title matching NLP result 

In [21]:
c_l = range(len(predicted_title)) 
sc_score_list_title = []
for i in c_l: 
    sc_score_list_title.append(max(sc_score_title[i]))
nlp_output_title = list(zip(c_l,sc_score_list_title,predicted_title))

c_df['title_matching_similarity'] = sc_score_list_title 
c_df['title_matching_allwyn'] = predicted_title

Drop title matching score <= 0.4

In [22]:
c_df = c_df.drop(c_df[c_df.title_matching_similarity < 0.4].index)

Job discription matching 

In [23]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(a_df['a_jd'])
X_train_counts.shape
count_vect.vocabulary_.get(u'algorithm')

index = count_vect.get_feature_names()

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

a_tfidf = X_train_tfidf.toarray()

clf = MultinomialNB().fit(X_train_tfidf, a_df['a_title'])

docs_new = c_df['c_jd']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

c_tfidf = X_new_tfidf.toarray()

# cosine similarity score
sc_score = cosine_similarity(c_tfidf,a_tfidf)
print(max(sc_score[11])) 

predicted = clf.predict(X_new_tfidf)

text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])
text_clf.fit(c_df['c_jd'], c_df['c_title'])

0.40549334318705804


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

Convert numpy module into regular list 

In [24]:
output = predicted.tolist()
c_l = range(len(output)) 
sc_score_list = []
for i in c_l: 
    sc_score_list.append(max(sc_score[i]))
nlp_output = list(zip(sc_score_list,output))

c_df['jd_matching_similarity'] = sc_score_list 
c_df['jd_matched_allwyn_title'] = output 

Drop title matching score < 0.1 

In [25]:
c_df = c_df.drop(c_df[c_df.jd_matching_similarity < 0.1].index) 

Consider min years of experience for software developer III 

In [26]:
mask = (c_df['c_min_year'] >5) & (c_df['jd_matched_allwyn_title'] == 'Software Developer I')
c_df['jd_matched_allwyn_title'][mask] = 'Software Developer III' 

<ipython-input-26-730d1838e305>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_df['jd_matched_allwyn_title'][mask] = 'Software Developer III'


Convert pandas df into list with index and calculate average cosine similarity score 

In [27]:
c_df_final = c_df.values.tolist()
c_df_final = [(x[0],x[4],x[5],x[6],x[7]) for x in c_df_final]

def Average(lst):
    return sum(lst) / len(lst)
  
title_avg = Average(sc_score_list_title)
jd_avg = Average(sc_score_list)
print("average title matching score =", title_avg, "average jd matching score =", jd_avg) 

average title matching score = 0.40727175161407825 average jd matching score = 0.34771955921646724


Merge NLP output with original calc data set into database 

In [28]:
sql_string3 = """
    CREATE TABLE IF NOT EXISTS c_temp(
        c_row_id                        INTEGER, 
        title_cosine_similarity         REAL,
        title_matching_allwyn           TEXT,
        jd_cosine_similarity            REAL, 
        jd_matching_allwyn              TEXT
        ); 
    """ 

c.execute("DROP TABLE IF EXISTS c_temp")
c.execute(sql_string3)
c.executemany("INSERT INTO c_temp VALUES(?,?,?,?,?);", c_df_final)
calc_con.commit()  

sql_string4 = """
    CREATE TABLE IF NOT EXISTS calc_final AS     
    SELECT calc.*, c.title_cosine_similarity, c.title_matching_allwyn, c.jd_cosine_similarity, c.jd_matching_allwyn
    FROM calc 
    INNER JOIN c_temp c
    ON calc.c_row_id = c.c_row_id;      
    """
c.execute("DROP TABLE IF EXISTS calc_final")
c.execute(sql_string4).fetchall()
calc_con.commit() 

In [29]:
sql_string6 = """
    CREATE TABLE IF NOT EXISTS predictive_modeling_tbl AS 
    SELECT c.c_row_id, c.c_labor, c.begin_date, c.end_date, c.c_edu_level, c.c_min_year_exp, c.c_current_year, c.c_next_year, c.c_second_year, ct.jd_matching_allwyn
    FROM CALC c
    INNER JOIN c_temp ct ON c.c_row_id = ct.c_row_id; 
    """
    
c.execute("DROP TABLE IF EXISTS predictive_modeling_tbl")
c.execute(sql_string6)

c.close()